## Load parameters and calculate quiescent luminosity, assign spectral class and  if not already

In [4]:
%matplotlib inline
import numpy as np
import pandas as pd
import warnings
import specmatchemp.library
import specmatchemp.plots as smplot
import glob
warnings.filterwarnings("ignore")

from f_6_7_8 import merged_spec_class, Mbol_to_Lum, kepler_luminosity, L_quieterr

cluster_dict = {'M67':['M67','M67', 15, 4.0, 850],
                'Ruprecht_147':['Ruprecht 147','Ruprecht_147', 30, 2.5, 300.],
                'M44':['M44','M44', 47, 0.73, 181.5],
                'M45':['M45','M45',55.,0.12,120.],
                'NGC_1647':['NGC 1647','NGC_1647',0.,0.145,550.],
                'Pleiades':['Pleiades','Pleiades',0.,0.100,136.]
               }
DM = lambda x: 5.*np.log10(float(x))-5. 

#------------------------------------------------
cluster = 'M67'
test = 'run_09'
#------------------------------------------------

basics = cluster_dict[cluster]
dm = DM(basics[4])
global dm
l = '/home/ekaterina/Documents/appaloosa/stars_shortlist/{0}/{0}_parameter.csv'.format(cluster)
print(glob.glob(l))

# if glob.glob(l) != []:
specs = pd.read_csv(l, index_col='EPIC')
# else:
#params = pd.read_csv(l, index_col='EPIC')
#print(params.columns)
wavmin, wavmax = 3480., 9700.
lib = specmatchemp.library.read_hdf(wavlim=[wavmin,wavmax])
specs = merged_spec_class(specs)
specs['L_quiet'] = np.nan
specs['L_quiet_bol'] = np.nan
specs['L_quiet_err'] =np.nan

print(specs.columns)
print(specs.head())
for oid in specs.index.values:
    specs.L_quiet_bol[oid] = Mbol_to_Lum(specs.Mbol[oid])
    specs.L_quiet[oid] = kepler_luminosity(specs['T'][oid],lib)
    specs.L_quiet_err[oid] = L_quieterr(specs.L_quiet[oid],
                                        specs.R_Rsun[oid],
                                        specs.R_Rsun_err[oid],
                                        specs['T'][oid],
                                        specs['T_err'][oid],
                                        lib)




['/home/ekaterina/Documents/appaloosa/stars_shortlist/M67/M67_parameter.csv']
Reading library from /home/ekaterina/.specmatchemp/library.h5
Index([ 'Unnamed: 0',        'gmag',        'rmag',        'imag',
              'zmag',        'ymag',           'J',           'H',
                 'K',     'RAJ2000',     'DEJ2000',  'spec_class',
                 'T',        'logT',         'BCv',          'Mv',
              'logL',         'B-V',       'Bt-Vt',         'V-G',
               'U-B',        'V-Rc',        'V-Ic',        'V-Ks',
               'J-H',         'H-K',       'Ks-W1',       'W1-W2',
             'W1-W3',       'W1-W4',        'Msun',      'logAge',
               'b-y',         'M_J',        'M_Ks',        'Mbol',
               'i-z',         'z-Y',      'R_Rsun',      '#SpT.1',
         'lum_class',         'u_g',         'g_r',         'r_i',
               'i_z',         'z_J',         'J_H',         'H_K',
          'Jmag_abs',          '55',     'L_quiet', 'L_q

KeyError: "['L_quiet2'] not in index"

In [5]:
r = specs.L_quiet_err/specs.L_quiet
r.head()

EPIC
211392354    0.259645
211392234    0.222862
211396893    0.259645
211389537    0.273473
211396493    0.235708
dtype: float64

In [6]:
from f_6_7_8 import interactive_CMD
crs = {'g_i':('gmag','imag'),
       'J_K':('J','K'),
       'r_z':('rmag','zmag'),
       'g_z':('gmag','zmag'),}

for key, c in crs.items():
    print(c[0],c[1])
    interactive_CMD(specs,cid1=c[0],cid2=c[1])
    input('Plot for {} done?'.format(key))
    


gmag imag
HU


KeyboardInterrupt: 

In [ ]:
outs=pd.Series()
import os
for i in [1,2,3,4]:
    file = '/home/ekaterina/Documents/appaloosa/stars_shortlist/share/temp/selected-data{}.txt'.format(i)
    if os.stat(file).st_size != 0:
        df = pd.read_csv(file,header=None,sep=' ').T.dropna(how='any')
        df = df.rename(index=str, columns={0:'outlier'})
        df.outlier = df.outlier.astype('int')
        s = df.outlier
        outs = pd.concat([outs,s],ignore_index=True)
outs = outs.drop_duplicates()
print(outs)
for key, c in crs.items():
    CMD(specs,cluster,cid1=c[0],cid2=c[1],color=key,ylim=(19,5),outliers=outs)
outs.to_csv('/home/ekaterina/Documents/appaloosa/stars_shortlist/{0}/{0}_outliers.csv'.format(cluster))

In [ ]:
print(specs.shape)
specs = specs.drop(list(outs))
print(specs.shape)

In [ ]:
specs = specs.dropna(how='any',subset=['T','L_quiet'])

In [ ]:
print(specs.shape)

In [ ]:
#intermediate save
specs.to_csv(l)

In [ ]:
categor = ['{}{}'.format(c,num) for c in list('OBAFGKM') for num in list(range(10))]
type_ = CategoricalDtype(categories=categor, ordered=True)
specs.spec_class.astype(type_)
s = specs.sort_values(by='spec_class')
spec_class_hist(s,cluster,type_)

In [ ]:
#remove all objects with spectral class hotter than F4
specs = specs[specs.spec_class > 'F3']
specs.shape
specs.to_csv(l)

In [ ]:
CMD(specs,cluster,cid1='gmag',cid2='imag',color='g_i',ylim=(21,8))#19,5
CMD(specs,cluster,cid1='J',cid2='K',color='J_K',ylim=(18,6))#14,5
CMD(specs,cluster,cid1='rmag',cid2='zmag',color='r_z',ylim=(20,8))#18,5
CMD(specs,cluster,cid1='gmag',cid2='zmag',color='g_z',ylim=(20,9))#18,5

In [ ]:

flarelist = pd.read_csv('/home/ekaterina/Documents/appaloosa/stars_shortlist/NGC_1647/results/run_01/NGC_1647_run_01_flarelist.txt',
    delimiter='\t')
flarelist = flarelist[['Object ID ',' Number of Flares ']]
flarelist = flarelist.rename(index=str, columns= {'Object ID ':'EPIC',' Number of Flares ':'counts'})
flarelist = flarelist.set_index('EPIC')
specs = specs.join(flarelist, how='left')
print(specs.head())


In [ ]:
specs.counts

In [ ]:
CMD_count(specs,cluster,cid1='gmag',cid2='zmag',color='g_z',ylim=(20,9))

In [ ]:
def energy(oid_list, cluster, test, specs):
    clusterenergy, clusterenergy_err = [], []
    for oid in oid_list:
        myflares, flux, flares = import_flares(cluster, test, oid)
        L_quiet = specs['L_quiet'].loc[oid]
        L_quiet_err = specs['L_quiet_err'].loc[oid]
        s_s = flares[['istart_no_sys_no_thruster','istop_no_sys_no_thruster']].dropna()
        tmyed, tenergy = [],[]
        for i,row in s_s.iterrows():
            ed, ederr = ED(row['istart_no_sys_no_thruster'],
                           row['istop_no_sys_no_thruster'],
                           flux.time,flux.flux_model,flux.flux_gap, flux.error)
            energy_err = energyerr(ederr,)
            if (ed != 'nan') & (ed > 0):
                clusterenergy.append(np.log10(ed*L_quiet))
                clusterenergy_err.append(energy_err)
                tenergy.append(np.log10(ed*L_quiet))
                tenergy_err.append(energy_err)
                tmyed.append(ed)
            else:
                clusterenergy.append(np.nan)
                tenergy.append(np.nan)
                tmyed.append(np.nan)
        flares['myed'] = tmyed
        flares['myenerg'] = tenergy
        flares['myenerg_err'] = tenergy_err
        write_flares(flares, cluster, test, oid, typ='flares')
    return clusterenergy

def energyerr(ed, ederr, L_quiet, L_quiet_err):
    return (ed * L_quiet_err) + (ederr * L_quiet) 

def FFD(oid, cluster, test, specs,color='red'):
    myflares, flux, flares = import_flares(cluster, test, oid)
    T = specs['T'].loc[oid]
    L_quiet = specs['L_quiet'].loc[oid]
    histAppa, edgesAppa = cum_hist(flares, flux, L_quiet,bins=5)
    binmids=[(x+edgesAppa[i])/2. for i,x in enumerate(edgesAppa[1:])]
    return plt.plot(binmids[1:],np.log10(histAppa[1:]),color=color,alpha=0.8,label='EPIC {}, T={}K'.format(oid,T))

def cum_hist_average(oid_list, cluster, test, specs):
    ED_, bins=[], 600
    for oid in oid_list:

        myflares, flux, flares = import_flares(cluster, test, oid)
        T = specs['T'].loc[oid]
        L_quiet = specs['L_quiet'].loc[oid]
        s_s = flares[['istart_no_sys_no_thruster','istop_no_sys_no_thruster']].dropna()
        tmyed, tED_ = [],[]
        for i,row in s_s.iterrows():
            ed, ederr = ED(row['istart_no_sys_no_thruster'],
                    row['istop_no_sys_no_thruster'],
                    flux.time,flux.flux_model,flux.flux_gap, flux.error)
            if (ed != 'nan') & (ed >0):
                ED_.append(np.log10(ed*L_quiet))
                tED_.append(np.log10(ed*L_quiet))
                tmyed.append(ed)
            else:
                ED_.append(0)
                tED_.append(0)
                tmyed.append(0)
        flares['myed'] = tmyed
        flares['myenerg'] =tED_
    print(flares.head())
    hist, edges=np.histogram(ED_, bins=bins)
    hist = np.flip(np.cumsum(np.flip(hist/oid_list.shape[0],axis=0)),axis=0)
    return hist, edges

def FFD_average(specs, cluster, test, class_='K7', 
                Tmin=3000., Tmax=7000., color='red',
                subset='class'):
    if subset == 'class':
        select = np.asarray([i for sublist in [specs[specs.spec_class.str.contains(c)].index.values 
                                               for c in class_] 
                                                 for i in sublist])
        label = '{}, n={}'.format(tuple(class_),len(select))
    elif subset == 'temperature':
        select = np.asarray([i for i in specs[(specs['T']>=Tmin) & (specs['T']<=Tmax)].index.values])
        label = '{}K - {}K, n={}'.format(Tmin,Tmax,len(select))
    left, right = 250, -160
    #select = specs[specs.spec_class.str.contains(class_)].index.values
    hist, edges = cum_hist_average(select, cluster, test, specs)
    binmids=[(x+edges[i])/2. for i,x in enumerate(edges[1:])]
    p,residuals, rank, singular_values, rcond= np.polyfit(x=binmids[left:right],y=np.log10(hist[left:right]),deg=1, full=True)
    print(p,residuals, rank, singular_values, rcond)
    plt.plot(binmids, p[0]*np.asarray(binmids)+p[1])
    plt.axvline(x=binmids[left])
    plt.axvline(x=binmids[right])
    plt.scatter(binmids,np.log10(hist),color=color,alpha=0.5,label=label)
    return p, left, right, binmids


plt.figure(figsize=(12,8))
plt.title('Cumulative FFD: {}, {}'.format(cluster,test))
#FFD_average(specs, cluster, test,class_=['K','M'],color='orange')
#FFD_average(specs, cluster, test,class_=['G'],color='green')
#FFD_average(specs, cluster, test,class_=['F'],color='blue')
#c_ = ['K','M']
#c_ = 'G'
#c_ = 'F'
c_ = ['F','G','K','M']
p,left, right, binmids = FFD_average(specs, cluster, test,
                                     class_=list(c_),color='green',
                                     Tmin=5000,Tmax=7000,
                                     subset='temperature')
p,left, right, binmids = FFD_average(specs, cluster, test,
                                     class_=list(c_),color='red',
                                     Tmin=3000,Tmax=4000,
                                     subset='temperature')
#p,left, right, binmids = FFD_average(specs, cluster, test,
                    #                 class_=list(c_),color='orange',
                       #              Tmin=4000,Tmax=5000,
                    #                 subset='temperature')
plt.legend()
plt.xlabel('log(energy in erg)')
plt.ylabel('log(frequency)')
if c_==['']: c_='all'
elif type(c_) == list: c_ = ''.join(map(str, c_))
plt.savefig('/home/ekaterina/Documents/appaloosa/stars_shortlist/{0}/results/{1}/plots/{0}_{1}_cum_FFD_{2}.jpg'.format(cluster,test,c_))


In [ ]:

specs['247135437'].head()

In [ ]:
if c_==['']: c_='all'
elif type(c_) == list: c_ = ''.join(map(str, c_))
print(c_)
columns = ['run','cluster','class','alpha','beta','cut_left','cut_right']
d = dict(zip(columns,[[test], [cluster],[c_],[-p[0]], [p[1]],[binmids[left]],[binmids[right]]]))
#tab = pd.DataFrame(d)
#tab = tab[columns]
tab = pd.read_csv('/home/ekaterina/Documents/appaloosa/stars_shortlist/share/compare_alpha.csv')#,index_col=0)
#print(tab)
#d = dict(zip(columns,[[test], [cluster],[c_],[-p[0]], [p[1]],[binmids[left]],[binmids[right]]]))
#tab = tab.append(pd.DataFrame(d))#,columns=columns)
#print(tab)
#tab.to_csv('/home/ekaterina/Documents/appaloosa/stars_shortlist/share/compare_alpha.csv')
tab

In [ ]:
import math
def find_nearest(array,value):
    idx = np.searchsorted(array, value, side="left")
    if idx > 0 and (idx == len(array) or math.fabs(value - array[idx-1]) < math.fabs(value - array[idx])):
        return idx-1
    else:
        return idx
    
cluster = 'M67'
info = pd.read_csv('/home/ekaterina/Documents/appaloosa/stars_shortlist/{}/results/tests.csv'.format(cluster))
class_ = 'FGKM'
r = tab[(tab['class'] == class_) & ((tab.run == 'run_06') | (tab.run == 'run_07') | (tab.run == 'run_04'))].sort_values(by='cluster')
plt.figure(figsize=(12,8))
plt.title('FFD power law fits to M67 and M44 for spectral class {}'.format(class_))

plt.xlabel('log(energy in erg)')
plt.ylabel('log(frequency)')
cc = {'M44':'blue','M67':'red'}
x = np.arange(31,36,0.05)
for idx, row in r.iterrows():
    iid = int(row.run[5:])-1
    l = find_nearest(x,row.cut_left)
    r = find_nearest(x,row.cut_right)
    plt.plot(x[l:r], 
             -row.alpha*x[l:r]+row.beta, 
             label='{}, N1 = {}, N2 = {}, N3 = {}, {}'.format(row.cluster, info.N1.loc[iid], info.N2.loc[iid], info.N3.loc[iid], info.other.loc[iid]),
             color=cc[row.cluster], alpha = float(row.run[5:])/4)
plt.legend()
#plt.savefig('/home/ekaterina/Documents/appaloosa/stars_shortlist/share/fits_{}_{}_comparison.jpg'.format(cluster,class_,))


In [ ]:
def color_negative_red(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    color = 'red' if val < 0 else 'black'
    return 'color: %s' % color


piv = pd.pivot_table(tab, values='alpha', index=['class','run'], columns=['cluster'])#, aggfunc=np.sum)
piv['M44-M67']=piv.M44-piv.M67
piv.style.applymap(color_negative_red)